# CFPB Consumer Complaints
Analysis of CFPB's public [Consumer Complaints Database] (http://www.consumerfinance.gov/data-research/consumer-complaints/)

**Visual Analysis**
* Complaints by company, issue, product, and tags
* Complaints over time
* Complaints by geographic region (state, zip code)

**NLP**
* Topic modeling on consumer narrative
* Topic modeling on company response

## Data Setup
---

In [1]:
#import statements
import numpy as np
import pandas as pd
import datetime
import urllib
 
from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict

In [2]:
# Access Complaints data via Socrata API
# source for connection code: https://dev.socrata.com/foundry/data.consumerfinance.gov/jhzv-w97w
query = ("https://data.consumerfinance.gov/resource/jhzv-w97w.json")
raw_data = pd.read_json(query)

In [4]:
# confirm data load
print raw_data.shape
raw_data.head()

(632107, 18)


,company,company_public_response,company_response,complaint_id,complaint_what_happened,consumer_consent_provided,consumer_disputed,date_received,date_sent_to_company,issue,product,state,sub_issue,sub_product,submitted_via,tags,timely,zip_code
0,Bank of America,Company chooses not to provide a public response,Closed with explanation,1780421,NaN,N/A,No,2016-02-09T16:58:05.000,2016-02-09T19:52:53.000,Problems caused by my funds being low,Bank account or service,NY,NaN,Checking account,Referral,NaN,Yes,11201
1,Synchrony Financial,NaN,Closed with explanation,1543488,XX/XX/XXXX I was approached by XXXX Attorney X...,Consent provided,No,2015-08-29T18:08:50.000,2015-09-04T14:55:10.000,Other fee,Credit card,TX,NaN,NaN,Web,NaN,Yes,787XX
2,Ocwen,NaN,Closed with explanation,1820018,I have been struggling with my mortgage paymen...,Consent provided,No,2016-03-08T01:29:34.000,2016-03-08T01:29:35.000,"Loan modification,collection,foreclosure",Mortgage,CA,NaN,Conventional fixed mortgage,Web,Older American,Yes,946XX
3,"SunTrust Banks, Inc.",NaN,Closed with explanation,629839,NaN,N/A,No,2013-12-11T17:05:57.000,2013-12-13T05:00:00.000,Cont'd attempts collect debt not owed,Debt collection,FL,Debt is not mine,I do not know,Phone,NaN,Yes,33056
4,"Select Portfolio Servicing, Inc",Company believes it acted appropriately as aut...,Closed with explanation,1774742,NaN,N/A,No,2016-02-05T19:36:58.000,2016-02-09T20:29:51.000,Other,Mortgage,CA,NaN,Other mortgage,Referral,NaN,Yes,92551


## Visual Analysis
---